In [25]:
import os
os.chdir("/home/niche/COLO/yolov9")
os.getcwd()
from pyniche.trainer import NicheTrainer
from pyniche.models.detection.yolo import NicheYOLO


In [26]:
DIR_DATA = "data/1a_angle_t2s"

In [27]:
trainer = NicheTrainer() 
trainer.type = "yolo"
trainer.set_data(
        dataclass=DIR_DATA,
        batch=16,
        n=128,
    )

In [29]:
!python3.9 train_dual.py \
--batch 16 --epochs 20 --img 640 --device 0 --min-items 0 --close-mosaic 15 \
--data data/1a_angle_t2s/data.yaml\
--weights yolov9-c.pt\
--cfg models/detect/yolov9-c.yaml\
--hyp data/hyps/hyp.scratch-high.yaml\
--project .\
--name out/yolov9/

^C
Traceback (most recent call last):
  File "/home/niche/COLO/yolov9/train_dual.py", line 12, in <module>
    import torch
  File "/home/niche/.conda/envs/tf/lib/python3.9/site-packages/torch/__init__.py", line 1476, in <module>
    from torch import func as func
  File "/home/niche/.conda/envs/tf/lib/python3.9/site-packages/torch/func/__init__.py", line 1, in <module>
    from torch._functorch.eager_transforms import (
  File "/home/niche/.conda/envs/tf/lib/python3.9/site-packages/torch/_functorch/eager_transforms.py", line 12, in <module>
    from torch.fx.experimental import const_fold
  File "/home/niche/.conda/envs/tf/lib/python3.9/site-packages/torch/fx/experimental/const_fold.py", line 6, in <module>
    from torch.fx.passes.split_module import split_module
  File "/home/niche/.conda/envs/tf/lib/python3.9/site-packages/torch/fx/passes/__init__.py", line 3, in <module>
    from . import net_min_base
  File "/home/niche/.conda/envs/tf/lib/python3.9/site-packages/torch/fx/passes/n

In [54]:
!python3.9 val_dual.py \
--weights out/yolov9/weights/best.pt\
--device 0\
--data data/1a_angle_t2s/data.yaml\
--task test\
--project .\
--name out/yolov9/eval\
--save-txt\
--save-conf\
--single-cls


val_dual: data=data/1a_angle_t2s/data.yaml, weights=['out/yolov9/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.45, iou_thres=0.7, max_det=300, task=test, device=0, workers=8, single_cls=True, augment=False, verbose=False, save_txt=True, save_hybrid=False, save_conf=True, save_json=False, project=., name=out/yolov9/eval, exist_ok=False, half=False, dnn=False, min_items=0
WARNING ⚠️ confidence threshold 0.45 > 0.001 produces invalid results
YOLOv5 🚀 2024-2-29 Python-3.9.16 torch-2.0.1+cu117 CUDA:0 (NVIDIA A100-SXM4-80GB, 81251MiB)

Fusing layers... 
yolov9-c summary: 604 layers, 50698278 parameters, 0 gradients, 236.6 GFLOPs
test: Scanning /home/niche/COLO/yolov9/data/1a_angle_t2s/test.cache... 50 images
                 Class     Images  Instances          P          R      mAP50   
                   all         50        709      0.749      0.354      0.576      0.373
Speed: 0.2ms pre-process, 10.9ms inference, 1.4ms NMS per image at shape (32, 3, 640, 640)
Results saved t

In [104]:
"""
A class to organize YOLO-structured data

Methods
---

get_images
    list of absolute paths of images in root/<split>/images
clone
    copy root/train and root/test to root/<folder_name>
shuffle_train_val
    shuffle self.ls_train_images and assign to
save_yaml
save_txt


Folder structure
---
root/
    train/ (required)
        images/
            img_1_13_jpg.rf.d69528304f2d10b633c6d94982185cb2.jpg
            img_2_13_jpg.rf.d69528304f2d10b633c6d94982185cb2.jpg
            ...
        labels/
            img_1_13_jpg.rf.d69528304f2d10b633c6d94982185cb2.txt
            img_2_13_jpg.rf.d69528304f2d10b633c6d94982185cb2.txt
            ...
    test/ (required)
        images/
            img_3.jpg
            img_4.jpg
            ...
        labels/
            img_3.txt
            img_4.txt

    test.txt (generated)
    train.txt (generated)
    val.txt (generated)
    data.yaml (generated)

Example YAML
---
path: /home/niche/cowsformer/data/cow200/yolov5/run3
train: "train.txt"
val: "val.txt"
test: "test.txt"
names:
  0: none
  1: cow

Example train.txt
---

/home/niche/cowsformer/data/cow200/yolov5/run0/images/img_32_jpg
/home/niche/cowsformer/data/cow200/yolov5/run0/images/img_1_jpg
/home/niche/cowsformer/data/cow200/yolov5/run0/images/img_1_26_jpg
/home/niche/cowsformer/data/cow200/yolov5/run0/images/img_1_62_jpg
/home/niche/cowsformer/data/cow200/yolov5/run0/images/img_1_10_jpg
/home/niche/cowsformer/data/cow200/yolov5/run0/images/img_3_11_jpg
/home/niche/cowsformer/data/cow200/yolov5/run0/images/img_4_3_jpg
"""

import os
import shutil
import random
import PIL
import torch
import numpy as np
import supervision as sv

# local imports
from pyniche.data.bbox import xywh2xyxy


class YOLO_API:
    def __init__(
        self,
        root: str,
    ):
        self.root = root
        self.ls_train_images_all = self.get_images("train")
        self.ls_train_images = None
        self.ls_val_iamges = None
        self.ls_test_images = self.get_images("test")
        self.save_txt("test")

    def get_PIL(self, split, idx):
        """
        get PIL image from split and idx

        params
        ------
        split: str
            "train" or "test"
        idx: int
            index of the image

        return
        ------
        PIL.Image
        """
        if split == "train":
            path = self.ls_train_images_all[idx]
        else:
            path = self.ls_test_images[idx]
        return PIL.Image.open(path)

    def get_images(self, split):
        """
        search images names (.jpg) in root/<split>/images

        params
        ------
        split: str
            "train" or "test"

        return
        ------
        a list of aboslute paths of images
        """
        return self.get_filepaths(split, "images")

    def get_labels(self, split):
        """
        search label names (.txt) in root/<split>/labels

        params
        ------
        split: str
            "train" or "test"

        return
        ------
        a list of aboslute paths of labels
        """
        return self.get_filepaths(split, "labels")

    def get_filepaths(self, split, folder):
        """
        get file paths of images or labels

        params
        ---
        split: str
            "train" or "test"
        folder: str
            "images" or "labels"
        """
        path_files = os.path.join(self.root, split, folder)
        ext = ".jpg" if folder == "images" else ".txt"
        ls_files = [f for f in os.listdir(path_files) if f.endswith(ext)]
        ls_files = [os.path.join(path_files, f) for f in ls_files]
        return sorted(ls_files)

    def get_detections(self, split, path_preds=None):
        """
        get sv.Detections from labels

        params
        ------
        split: str
            "train" or "test"
        path_results: str
            path to the dir of predictions (.txt). If provided, the detections will be
            created from the predictions, and the format will be
            [class_id, x_center, y_center, width, height, confidence].
            Otherwise, from the labels.

        return
        ------
        a list of sv.Detections
        """
        detections = []
        # get file paths
        if path_preds:
            labels = [f for f in os.listdir(path_preds) if f.endswith(".txt")]
            labels = sorted([os.path.join(path_preds, f) for f in labels])
        else:
            labels = self.get_labels(split)
        images = self.get_images(split)
        n_samples = len(images)
        # iterate each pair of image and label
        for i in range(n_samples):
            # get image info
            image = PIL.Image.open(images[i])
            img_w, img_h = image.size
            # get annotation
            label = labels[i]
            with open(label, "r") as f:
                lines = f.readlines()
                lines = [l.strip() for l in lines]
            # each detection in the image/label
            ls_xyxy = []
            ls_cls = []
            ls_conf = []
            for l in lines:
                parts = l.split(" ")
                class_id = int(parts[0])
                coords = tuple(
                    map(float, parts[1:5])
                )  # x_center, y_center, width, height
                conf = float(parts[5]) if path_preds else None
                xyxy = xywh2xyxy(
                    coords,
                    img_size=(img_w, img_h),
                )
                # append to lists
                ls_xyxy.append(xyxy)
                ls_cls.append(class_id)
                if path_preds:
                    ls_conf.append(conf)
            # create sv.Detections
            ls_xyxy = torch.stack(ls_xyxy).numpy()
            ls_cls = np.array(ls_cls)
            ls_conf = np.array(ls_conf)
            detection = sv.Detections(
                ls_xyxy,
                class_id=ls_cls,
                confidence=ls_conf if path_preds else None,
            )
            detections.append(detection)
        return detections

    def clone(self, folder_name):
        """
        copy root/train and root/test to
        root/<folder_name>/train and root/<folder_name>/test
        """
        path_train = os.path.join(self.root, "train")
        path_test = os.path.join(self.root, "test")
        path_folder = os.path.join(self.root, folder_name)
        if os.path.exists(path_folder):
            shutil.rmtree(path_folder)
        os.mkdir(path_folder)
        shutil.copytree(path_train, os.path.join(path_folder, "train"))
        shutil.copytree(path_test, os.path.join(path_folder, "test"))
        # copy yaml and other txt
        shutil.copy(os.path.join(self.root, "data.yaml"), path_folder)
        shutil.copy(os.path.join(self.root, "train.txt"), path_folder)
        shutil.copy(os.path.join(self.root, "test.txt"), path_folder)

    def shuffle_train_val(self, n=None, k=5):
        """
        shuffle self.ls_train_images and assign to
        self.ls_train_images and self.ls_val_images

        params
        ------
        n: None or int or float
            None: use all images
            int: number of images to be included in the train/val set
            float: ratio of images to be included in the train/val set
        k: int
            how many folds to split the train/val set
        """
        # determine n
        total_n = len(self.ls_train_images_all)
        if n is None:
            n = total_n
        elif isinstance(n, float):
            n = int(n * total_n)
        n_val = int(n / k)
        # shuffle training images
        random.shuffle(self.ls_train_images_all)
        train_images = self.ls_train_images_all[:n]
        # split train/val
        self.ls_train_images = train_images[:-n_val]

        self.ls_val_images = train_images[-n_val:]
        self.save_txt("train")
        self.save_txt("val")

    def save_yaml(self, classes, name="data.yaml"):
        """
        make data.yaml in root

        params
        ------
        classes: list
            e.g., ["cow", "none"]

        name: str
            name of the yaml file

        """
        path_yaml = os.path.join(self.root, name)
        with open(path_yaml, "w") as f:
            f.write(f"path: {self.root}\n")
            f.write(f'train: "train.txt"\n')
            f.write(f'val: "val.txt"\n')
            f.write(f'test: "test.txt"\n')
            f.write("names:\n")
            for i, c in enumerate(classes):
            t    f.write(f"  {i}: {c}\n")

    def save_txt(self, split):
        """
        save <split>.txt in root
        """
        path_txt = os.path.join(self.root, f"{split}.txt")
        with open(path_txt, "w") as f:
            for img in getattr(self, f"ls_{split}_images"):
                f.write(img + "\n")

    def path_yaml(self):
        return os.path.join(self.root, "data.yaml")


array(None, dtype=object)

In [106]:
# from pyniche.data.yolo.API import YOLO_API
api = YOLO_API(DIR_DATA)
lbs = api.get_detections("test")
pre = api.get_detections("test", path_preds="out/yolov9/eval2/labels")

In [108]:
from pyniche.evaluate import from_sv
out = from_sv(pre, lbs)
print(out)

{'map5095': 0.3725151422749049, 'map50': 0.5756328291933566, 'precision': 0.7731343283582089, 'recall': 0.36530324400564174, 'f1': 0.4961685823754789, 'n_all': 50, 'n_fn': 450, 'n_fp': 76}
